In [ ]:
import pandas as pd
import os
import re
from sklearn.metrics import classification_report
final_ans = {'method':[], 'backbone': [], 'ratio': [], 'dataset': [], 'metric':[], 'values': []}
for dataset in ['clinc', 'banking', 'stackoverflow', 'ele', 'news', 'thucnews']:
    for ratio in [0.25, 0.5, 0.75]:
        for method in ['analogy', 'simple']:
            for backbone in ['llama8b', 'qwen27B', 'deepseek-v3:671b' ]:
                if not os.path.exists(f'outputs/results/{method}/{backbone}_{dataset}_{ratio}.tsv'):
                    continue
                df = pd.read_csv(f'outputs/results/{method}/{backbone}_{dataset}_{ratio}.tsv', sep='\t')
                df = df[~df['pred_label'].isna()]
                label_list = pd.read_csv(f'../../data/{dataset}/{dataset}_{ratio}.txt', header=None)[0].tolist() + ['OOD']
                df['gt_label'] = df['gt_label'].apply(lambda x: x if x in label_list else 'OOD')
                df['pred_label'] = df['pred_label'].apply(lambda x: re.findall("|".join(label_list), x)[0] if len(re.findall("|".join(label_list), x)) > 0 else 'OOD')
                # df['pred_label'] = df['pred_label'].apply(lambda x: x if x in label_list else 'OOD')
                
                ans = classification_report(df['gt_label'], df['pred_label'], output_dict=True)
                metrics = {
                    "F1-Score": ans['macro avg']['f1-score'],
                    "ACC": ans['accuracy'],
                    "K-F1": sum([ans[i]['f1-score'] for i in ans if i != 'OOD' and i in label_list]) / len([ans[i]['f1-score'] for i in ans if i != 'OOD' and i in label_list]),
                    "N-F1": ans['OOD']['f1-score']
                }
                for metric, values in metrics.items():
                    final_ans['method'].append(method)
                    final_ans['backbone'].append(backbone)
                    final_ans['ratio'].append(ratio)
                    final_ans['dataset'].append(dataset)
                    final_ans['metric'].append(metric)
                    final_ans['values'].append(values)
df = pd.DataFrame(final_ans)
df.to_csv('exp.csv', index=None)

/ssd/chenxi/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ssd/chenxi/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ssd/chenxi/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [ ]:
df = pd.read_csv('exp.csv')
df_pivot = df.pivot(index=['dataset', 'backbone', 'method'], columns=['ratio', 'metric'], values=['values'])
df_pivot

values                      \
ratio                                       0.25                       
metric                                  F1-Score       ACC      K-F1   
dataset       backbone         method                                  
banking       llama8b          analogy  0.569134  0.811891  0.582522   
                               simple   0.454063  0.499350  0.471377   
clinc         deepseek-v3:671b analogy  0.813460  0.880973  0.810635   
                               simple   0.592748  0.551111  0.592756   
              llama8b          analogy  0.619693  0.819556  0.612613   
                               simple   0.467332  0.334667  0.471396   
stackoverflow llama8b          analogy  0.597099  0.835420  0.536484   
                               simple   0.530675  0.598231  0.501013   

                                                                      \
ratio                                                 0.50             
metric                                      N-F1  F1-Score       ACC   
dataset       backbone         method                                  
banking       llama8b          analogy  0.883900  0.287435  0.570175   
                               simple   0.579169  0.445883  0.492853   
clinc         deepseek-v3:671b analogy  0.917982  0.835882  0.860124   
                               simple   0.592471  0.645842  0.455313   
              llama8b          analogy  0.881645  0.566127  0.697778   
                               simple   0.316966  0.519531  0.459556   
stackoverflow llama8b          analogy  0.900173       NaN       NaN   
                               simple   0.678984  0.703781  0.653673   

                                                                      \
ratio                                                           0.75   
metric                                      K-F1      N-F1  F1-Score   
dataset       backbone         method                                  
banking       llama8b          analogy  0.284197  0.697899  0.447205   
                               simple   0.454674  0.557721  0.522384   
clinc         deepseek-v3:671b analogy  0.835347  0.875989  0.835887   
                               simple   0.650712  0.280597  0.327581   
              llama8b          analogy  0.563473  0.765157  0.476189   
                               simple   0.521199  0.394441  0.538347   
stackoverflow llama8b          analogy       NaN       NaN       NaN   
                               simple   0.711432  0.627269       NaN   

                                                                      
ratio                                                                 
metric                                       ACC      K-F1      N-F1  
dataset       backbone         method                                 
banking       llama8b          analogy  0.479857  0.446677  0.477297  
                               simple   0.493177  0.535005  0.325409  
clinc         deepseek-v3:671b analogy  0.820524  0.836778  0.736111  
                               simple   0.192000  0.330117  0.043624  
              llama8b          analogy  0.520444  0.475965  0.501276  
                               simple   0.496889  0.540637  0.281863  
stackoverflow llama8b          analogy       NaN       NaN       NaN  
                               simple        NaN       NaN       NaN